<a href="https://colab.research.google.com/github/skyloma/img2/blob/master/ColabAndTpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 授权访问google cloud partform

In [0]:
from google.colab import auth
auth.authenticate_user()
project_id = 'dreamer8090'
!gcloud config set project {project_id}

# github上加载models

In [0]:
!git clone https://github.com/tensorflow/models.git  
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

#This creates a name_pb2.py file from every name.proto file in the \object_detection\protos folder.
!cd models/research; protoc object_detection/protos/*.proto --python_out=.

# 切换到目录research下进行训练

In [0]:
from tensorboardcolab import TensorBoardColab
tbc=TensorBoardColab(graph_path='gs://staging.dreamer8090.appspot.com/tpu128')

In [0]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/model_tpu_main.py \
--gcp_project=dreamer8090  \
--pipeline_config_path=tpu128.config \
--model_dir=gs://staging.dreamer8090.appspot.com/tpu128 \
--num_train_steps=100000 \
--num_eval_steps=8000

# 导出能转换成tflte的pb

In [0]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=tpu128.config \
--trained_checkpoint_prefix=gs://staging.dreamer8090.appspot.com/data/tpu128/model.ckpt-50000 \
--output_directory=gs://staging.dreamer8090.appspot.com/outmodel/tpu128 \
--add_postprocessing_op=true

# 转换量化的tflite

In [0]:
!tflite_convert \
  --graph_def_file=/content/outmodel/tpu128/tflite_graph.pb \
  --output_format=TFLITE \
  --output_file=/content/outmodel/tpu128/tflite_graph.tflite \
  --input_arrays=normalized_input_image_tensor \
  --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
  --input_shapes=1,128,128,3 \
  --inference_type=QUANTIZED_UINT8 \
  --mean_value=128 \
  --std_dev_values=128 \
  --default_ranges_min=0 \
  --default_ranges_max=6 \
  --change_concat_input_ranges=false \
  --allow_custom_ops
 

# 下载模型到mytf4930工程 解压copy到gs://mytf_storage/data/

In [0]:
# cd /tmp
!curl -O http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
!tar xzf ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz

!gsutil cp -r ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/ gs://staging.dreamer8090.appspot.com/data/

In [0]:
!git clone https://github.com/skyloma/img2.git  

In [0]:
!cd models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python xml_to_csv.py   --image_dir=/content/img2/

In [0]:
%cd /content  
!ls


/content


In [0]:
import os
os.chdir("models/research")

In [0]:
! export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim;python object_detection/generate_tfrecord.py \
    --csv_input=/content/img2/train_labels.csv \
    --image_dir=/content/img2/train \
    --output_path=/content/data/train.record  
!  export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/generate_tfrecord.py \
    --csv_input=/content/img2/test_labels.csv \
    --image_dir=/content/img2/test \
    --output_path=/content/data/test.record  

In [0]:
%cd /content  
!ls

In [0]:


!gsutil cp -r /content/data {bucket_name}
!gsutil cp -r /content/models/research/label/ {bucket_name}/data/ 


# 导出训练生成的文件

In [0]:
bucket_name="gs://staging.dreamer8090.appspot.com"
!gsutil cp  -r gs://staging.dreamer8090.appspot.com/tpu128  outdata/
  
# !gsutil cp  gs://{bucket_name}/data/pk_rmb.pbtxt A/
  
# !gsutil cp  gs://{bucket_name}/A/tflite_graph.pbtxt A/  